<a href="https://colab.research.google.com/github/prinzessinmarlenifee/SenseCap/blob/main/SenseCap_v4_dataaugment_global_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Versuch 3: Single sheets ohne sync - Global Model



Saveing model and checkpoints for Zwischenspeicherung

CSV-sensor data sheet
1.   wrist
2.   seat
3.  head
-> 3 Sensoren, 9 Spalten
-> checkt erste und zweite Zeile für header (da unterschiedlich)

*   hot encoding for labeling
 Action, Pedaling, Resting, Pushing

 Summary at end with


*   test accuracy
*   confusion matrix





all data is stored in Drive



##Model Choice
✅ Teil 2: Ist CNN + LSTM eine gute Architektur für IMU-Zeitreihendaten?

Ja, das ist eine bewährte Kombination – besonders bei IMU-Daten (z. B. Accelerometer/Gyroscope), weil:
🔎 Warum CNN?

    Erkennt lokale Muster in kurzen Zeitfenstern (z. B. Bewegungsphasen)

    Spart Rechenzeit, da es weniger Parameter hat als ein reines LSTM

🔁 Warum LSTM?

    Erkennt zeitliche Abhängigkeiten (z. B. Bewegungsabfolgen)

    Ideal für sequentielle Daten, wie du sie hast

✅ Alternativen oder Erweiterungen

Falls du später mehr Leistung brauchst:

    Bidirectional LSTM → besser für symmetrische Bewegungsabfolgen

    Residual CNN Blocks → für tiefere Netzwerke

    Transformer → wenn du sehr viele Daten und lange Sequenzen hast

In [9]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')


ValueError: Mountpoint must not already contain files

In [6]:
#import packages
import os
import json
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import tensorflow as tf

layers = tf.keras.layers
models = tf.keras.models


In [7]:
# --- PARAMETER ---
sampling_rate = 60       # 60 Hz nach SDI-Algorithmus
window_size = 60         # 1 Sekunde = 60 Frames
step_size = 30           # 50% Überlappung

# Basisverzeichnis (sollte die 18 Session-Ordner enthalten)
base_dir = '/content/drive/MyDrive/ML-MTB-Modell/IMU-Sessions/'


#Sessions einlesen

In [8]:
session_dirs = sorted([
    d for d in os.listdir(base_dir)
    if os.path.isdir(os.path.join(base_dir, d))
])
print(f"Gefundene Sessions: {len(session_dirs)} -> {session_dirs}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ML-MTB-Modell/IMU-Sessions/'

Fuktionen zum LAbel-Parsing und Datei finden

In [ ]:
# 5.1 parse_hot_labels: Liest die _hot.json-Datei ein, erstellt für jeden Frame ein Label
def parse_hot_labels(json_path, total_frames):
    with open(json_path, 'r') as f:
        data = json.load(f)
    entries = data['button_presses'].strip().split(';')

    label_changes = []
    for entry in entries:
        if ':' in entry:
            label, frame = entry.strip().split(':')
            label = label.strip()
            # Korrigiere evtl. "Peadling" → "Pedaling"
            if label.lower() == 'peadling':
                label = 'Pedaling'
            label_changes.append((int(frame.strip()), label))

    frame_labels = ['Unknown'] * total_frames
    for i, (start_frame, label) in enumerate(label_changes):
        end_frame = label_changes[i + 1][0] if i + 1 < len(label_changes) else total_frames
        for f in range(start_frame, min(end_frame, total_frames)):
            frame_labels[f] = label
    return frame_labels

# 5.2 find_sensor_file: Findet CSV-Datei, deren Name mit dem Prefix beginnt (Head_, Wrist_, Seat_)
def find_sensor_file(folder, prefix):
    for f in os.listdir(folder):
        if f.lower().startswith(prefix.lower()):
            return os.path.join(folder, f)
    raise FileNotFoundError(f"❌ Keine Datei mit Prefix '{prefix}' in {folder} gefunden.")

# 5.3 find_hot_file: Findet JSON-Datei, deren Name auf "_hot.json" endet
def find_hot_file(folder):
    for f in os.listdir(folder):
        if f.lower().endswith('_hot.json'):
            return os.path.join(folder, f)
    raise FileNotFoundError(f"❌ Keine Datei mit Suffix '_hot.json' in {folder} gefunden.")


# 6. Fensterung für drei Sensoren kombinieren

In [ ]:
def window_data_multiple_sensors(head_data, wrist_data, seat_data, frame_labels):
    X_windows, y_windows = [], []
    total_frames = len(frame_labels)

    # Annahme: head_data, wrist_data, seat_data haben alle dieselbe Anzahl Zeilen = total_frames
    for start in range(0, total_frames - window_size + 1, step_size):
        end = start + window_size

        win_h = head_data[start:end]    # (window_size, 6)
        win_w = wrist_data[start:end]
        win_s = seat_data[start:end]
        window = np.concatenate([win_h, win_w, win_s], axis=1)  # → (window_size, 18)

        label_window = frame_labels[start:end]
        dominant_label = Counter(label_window).most_common(1)[0][0]
        if dominant_label == 'Unknown':
            continue

        X_windows.append(window)
        y_windows.append(dominant_label)

    return np.array(X_windows), np.array(y_windows)


#7. Daten einlesen und Fenster / Labels erzeugen
→ Nach Ausführung siehst du für jede Session etwa: “→ 153 Fenster, 3 Klassen” etc.

In [ ]:
#Vorbereitung
sessions_X = []
sessions_y = []
valid_sessions = []  # <- neue Liste! mit nur valid sessions
skipped_sessions = []

#features definieren:
features = ['Euler_X', 'Euler_Y', 'Euler_Z', 'Acc_X', 'Acc_Y', 'Acc_Z', 'Gyr_X', 'Gyr_Y', 'Gyr_Z']

#Funktionen für Daten einlesen
def smart_feature_filter(df):
    # alles lowercase und leerzeichenfrei vergleichen
    keep = [col for col in df.columns if any(kw in col.lower() for kw in ['euler', 'acc', 'gyr'])]
    return df[keep]

def print_csv_headers(path, label):
    with open(path, 'r') as f:
        lines = f.readlines()
    print(f"\n📋 {label} → Datei: {os.path.basename(path)}")
    print("   Zeile 0:", lines[0].strip())
    print("   Zeile 1:", lines[1].strip())


def inspect_sensor_csv(path):
    df = pd.read_csv(path, sep=',', skiprows=1)
    df.columns = df.columns.str.strip()
    print(f"📊 {os.path.basename(path)}: {df.shape[1]} Spalten")
    print("   → Spaltennamen:", df.columns.tolist())




def load_sensor_csv(path):
  import csv

  # Erste zwei Zeilen lesen
  with open(path, 'r') as f:
      reader = csv.reader(f)
      first_line = next(reader)
      second_line = next(reader)

  # Prüfen ob erste Zeile ein Header ist (z. B. mit bekannten Schlüsselwörtern)
  first_line_str = ",".join(first_line).lower()
  if any(kw in first_line_str for kw in ['euler', 'acc', 'gyr']):
      skip = 0
  else:
      skip = 1

  # Einlesen
  df = pd.read_csv(path, sep=',', skiprows=skip)
  df.columns = df.columns.str.strip()

  # Features filtern
  df = smart_feature_filter(df)

  # Numerisch umwandeln und NaN behandeln
  df = df.apply(pd.to_numeric, errors='coerce').fillna(0).astype(np.float32)

  return df.values

  print(f"🔍 {os.path.basename(path)}: Header {'erste Zeile' if skip==0 else 'zweite Zeile'}")




# --- Hauptschleife ---
for sess_dir in session_dirs:
    print(f"\n📂 Lade Session: {sess_dir}")
    session_path = os.path.join(base_dir, sess_dir)


    # 7.1 Sensor-Dateien finden
    head_path  = find_sensor_file(session_path, 'Head_')
    wrist_path = find_sensor_file(session_path, 'Wrist_')
    seat_path  = find_sensor_file(session_path, 'Seat_')

    # 7.2 Hot-JSON-Datei finden
    hot_path = find_hot_file(session_path)

    inspect_sensor_csv(head_path)
    inspect_sensor_csv(wrist_path)
    inspect_sensor_csv(seat_path)


    #print csv-heads for debugging & checking (oben definierte function print_csv_headers)
    #print_csv_headers(head_path, 'Head')
    #print_csv_headers(wrist_path, 'Wrist')
    #print_csv_headers(seat_path, 'Seat')




    # 7.3 IMU-Daten laden
    #funktion um imu laden
    #aktuelles Problem: header in der zweiten Zeile, seperator ',' , erkennt nur zwei spalten beim einlesen



    head_data  = load_sensor_csv(head_path)
    wrist_data = load_sensor_csv(wrist_path)
    seat_data  = load_sensor_csv(seat_path)




    print(f"📊 Sensorlängen: Head={head_data.shape}, Wrist={wrist_data.shape}, Seat={seat_data.shape}")

    #expected_features = 27  # 3 Sensoren × 9 Features (oben definiert)
   # if X_win.shape[1:] != (window_size, expected_features):
    #      print(f"⚠️ Session {sess_dir} hat Format {X_win.shape[1:]}, wird übersprungen.")
     #     skipped_sessions.append(sess_dir)
      #    continue



    #7.4 Labels laden
    total_frames = min(head_data.shape[0], wrist_data.shape[0], seat_data.shape[0])
    frame_labels = parse_hot_labels(hot_path, total_frames)



    # 7.5 Sicherheitskürzung (später optional mit synch.json ersetzen)
    head_data  = head_data[:total_frames]
    wrist_data = wrist_data[:total_frames]
    seat_data  = seat_data[:total_frames]
    frame_labels = frame_labels[:total_frames]


    # 7.6 Fensterung & Label-Zuweisung
    X_win, y_win = window_data_multiple_sensors(head_data, wrist_data, seat_data, frame_labels)

    # 5. Gültigkeit prüfen
    if len(X_win) == 0:
        print(f"⚠️  Session {sess_dir} übersprungen – keine gültigen Fenster.")
        skipped_sessions.append(sess_dir)
        continue

    expected_features = 27  # oder dynamisch aus den Daten

    if X_win.shape[1:] != (window_size, expected_features):
        print(f"⚠️  Session {sess_dir} hat Format {X_win.shape[1:]}, wird übersprungen.")
        skipped_sessions.append(sess_dir)
        continue


        # 6. Speichern
    sessions_X.append(X_win)
    sessions_y.append(y_win)
    valid_sessions.append(sess_dir)
    print(f"✅ Session {sess_dir}: {len(X_win)} Fenster, {len(np.unique(y_win))} Klassen")

# --- Zusammenfassung ---
print("\n✅ Verwendete Sessions:")
for idx, sess in enumerate(valid_sessions):
    print(f"  {sess}: {sessions_X[idx].shape}")

if skipped_sessions:
    print("\n⛔️ Übersprungene Sessions:")
    for s in skipped_sessions:
        print(f"  {s}")











📂 Lade Session: Session_01
📊 Head_D422CD00563B_20230713_082527.csv: 12 Spalten
   → Spaltennamen: ['PacketCounter', 'SampleTimeFine', 'Euler_X', 'Euler_Y', 'Euler_Z', 'Acc_X', 'Acc_Y', 'Acc_Z', 'Gyr_X', 'Gyr_Y', 'Gyr_Z', 'Unnamed: 11']
📊 Wrist_D422CD004550_20230713_082527.csv: 12 Spalten
   → Spaltennamen: ['PacketCounter', 'SampleTimeFine', 'Euler_X', 'Euler_Y', 'Euler_Z', 'Acc_X', 'Acc_Y', 'Acc_Z', 'Gyr_X', 'Gyr_Y', 'Gyr_Z', 'Unnamed: 11']
📊 Seat_D422CD00456D_20230713_082527.csv: 12 Spalten
   → Spaltennamen: ['PacketCounter', 'SampleTimeFine', 'Euler_X', 'Euler_Y', 'Euler_Z', 'Acc_X', 'Acc_Y', 'Acc_Z', 'Gyr_X', 'Gyr_Y', 'Gyr_Z', 'Unnamed: 11']
📊 Sensorlängen: Head=(60077, 9), Wrist=(60078, 9), Seat=(60075, 9)
✅ Session Session_01: 1997 Fenster, 3 Klassen

📂 Lade Session: Session_02
📊 Head_D422CD00563B_20230713_085629.csv: 12 Spalten
   → Spaltennamen: ['PacketCounter', 'SampleTimeFine', 'Euler_X', 'Euler_Y', 'Euler_Z', 'Acc_X', 'Acc_Y', 'Acc_Z', 'Gyr_X', 'Gyr_Y', 'Gyr_Z', 'Unnamed:

<ipython-input-7-bde1625cbf8d>:25: DtypeWarning: Columns (5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=',', skiprows=1)


📊 Wrist_D422CD004550_20230727_073528.csv: 12 Spalten
   → Spaltennamen: ['PacketCounter', 'SampleTimeFine', 'Euler_X', 'Euler_Y', 'Euler_Z', 'Acc_X', 'Acc_Y', 'Acc_Z', 'Gyr_X', 'Gyr_Y', 'Gyr_Z', 'Unnamed: 11']


<ipython-input-7-bde1625cbf8d>:25: DtypeWarning: Columns (5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=',', skiprows=1)


📊 Seat_D422CD00456D_20230727_073528.csv: 12 Spalten
   → Spaltennamen: ['PacketCounter', 'SampleTimeFine', 'Euler_X', 'Euler_Y', 'Euler_Z', 'Acc_X', 'Acc_Y', 'Acc_Z', 'Gyr_X', 'Gyr_Y', 'Gyr_Z', 'Unnamed: 11']


<ipython-input-7-bde1625cbf8d>:50: DtypeWarning: Columns (5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=',', skiprows=skip)
<ipython-input-7-bde1625cbf8d>:50: DtypeWarning: Columns (5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=',', skiprows=skip)


📊 Sensorlängen: Head=(65763, 9), Wrist=(65763, 9), Seat=(65763, 9)
✅ Session Session_11: 2181 Fenster, 4 Klassen

📂 Lade Session: Session_12
📊 Head_D422CD004576_20230801_075834.csv: 12 Spalten
   → Spaltennamen: ['0', '416010770', '-4.522172451019287', '48.04213333129883', '21.0351619720459', '0', '0.1', '0.2', '0.3', '0.4', '0.5', '']
📊 Wrist_D422CD004550_20230801_075834.csv: 12 Spalten
   → Spaltennamen: ['0', '532295258', '-42.42717361450195', '45.996009826660156', '-55.192440032958984', '0', '0.1', '0.2', '0.3', '0.4', '0.5', '']
📊 Seat_D422CD00456D_20230801_075834.csv: 11 Spalten
   → Spaltennamen: ['0', '409881162', '2202328491210930', '-6035955047607420', '-8357672119140620', '0.00000000000000', '0.00000000000000.1', '0.00000000000000.2', '0.00000000000000.3', '0.00000000000000.4', '0.00000000000000.5']
📊 Sensorlängen: Head=(59523, 9), Wrist=(59524, 9), Seat=(59514, 9)
✅ Session Session_12: 1967 Fenster, 4 Klassen

📂 Lade Session: Session_13
📊 Head_D422CD004576_20230802_080027.c

🧠 Dann ist deine Fensterform:

    3 Sensoren × 9 Spalten = 27 Features
    → Fenster-Shape: (window_size, 27) = (60, 27)


Euler_X, Euler_Y, Euler_Z


Acc_X, Acc_Y, Acc_Z

Gyr_X, Gyr_Y, Gyr_Z

→ = 9 physikalisch sinnvolle Spalten pro Sensor

#8. Global Model: Training and Evaluation

Schritt-für-Schritt Erklärung

  **1. Daten zusammenführen**

    Alle Sessions werden zu einem großen Datensatz verbunden, damit das Modell aus allen Beispielen lernt.

 **2. Label-Encoding**


    Die Labels (z.B. verschiedene Aktivitäten) werden in numerische Werte umgewandelt, da das Modell nur mit Zahlen arbeitet.

 **3. Trainings- und Validierungs-Split**

    Der Datensatz wird in Training (90%) und Validation (10%) aufgeteilt. Validation wird genutzt, um das Modell während des Trainings zu überprüfen.

  **4. Modell erstellen**

    Das Modell kombiniert Convolutional Layers (um lokale Muster in den Zeitreihen zu erkennen) mit LSTM (um zeitliche Abhängigkeiten zu lernen). Dropout wird eingesetzt, um Überanpassung zu vermeiden.

  **5. Checkpoint Callback**

    Während des Trainings werden Modelle nach jeder Epoche gespeichert (nur die besten, basierend auf Validierungsleistung).

  **6. Training**
  
    Das Modell lernt über 50 Epochen, wobei Trainings- und Validierungsdaten genutzt werden.

  **7. Finales Modell speichern**

    Das finale Modell wird nach dem Training gespeichert,
    um es später laden und nutzen zu können.

  **8. Evaluation auf Validierungsdaten**

    Das Modell wird auf den Validation-Daten getestet,
    um die Genauigkeit zu ermitteln.

  **9.  Vorhersagen erzeugen**

    Die Wahrscheinlichkeiten für jede Klasse werden ermittelt und in Klassen umgewandelt (höchste Wahrscheinlichkeit = Vorhersage).

  **10.  Labels prüfen**

    Nur die tatsächlich in den Validierungsdaten vorhandenen Klassen werden für den Bericht verwendet.

 **11.  Classification Report**

    Präzision, Recall, F1-Score und Support für jede Klasse werden ausgegeben — wichtige Kennzahlen für die Modellgüte.

  **12.  Confusion Matrix**

    Visualisiert die Fehler des Modells,
    zeigt welche Klassen oft verwechselt werden.



In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.utils.multiclass import unique_labels

# 💾 Speicherpfade anlegen
checkpoint_dir = "/content/drive/MyDrive/mtb_project/checkpoints_global"
final_model_dir = "/content/drive/MyDrive/mtb_project/final_models_global"
report_dir = "/content/drive/MyDrive/mtb_project/reports_global"
os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(final_model_dir, exist_ok=True)
os.makedirs(report_dir, exist_ok=True)

# 🔁 Augmentierungsfunktion
def augment_data(X, y, noise_std=0.01, time_shift_max=5):
    X_aug, y_aug = [], []
    for xi, yi in zip(X, y):
      # 1. Leichtes Rauschen hinzufügen
        noisy = xi + np.random.normal(0, noise_std, xi.shape)
      # 2. Zufälliges Zeitverschieben
        shift = np.random.randint(-time_shift_max, time_shift_max)
        shifted = np.roll(noisy, shift, axis=0)
        # Optional: Padding beim Rollen (statt wrap-around)
        if shift > 0:
            shifted[:shift, :] = 0
        elif shift < 0:
            shifted[shift:, :] = 0
        X_aug.append(shifted)
        y_aug.append(yi)
    return np.array(X_aug), np.array(y_aug)




# 1️⃣ Sessions zusammenführen (dieser Teil muss mit deinen Daten ergänzt werden!)
# Beispiel:
X_all = np.concatenate(sessions_X)
y_all = np.concatenate(sessions_y)

# 2️⃣ Labels encoden
le = LabelEncoder()
y_all_enc = le.fit_transform(y_all)

# 3️⃣ Train/Val-Split
X_train, X_val, y_train, y_val = train_test_split(
    X_all, y_all_enc, test_size=0.1, random_state=42, stratify=y_all_enc
)

# 4️⃣ Augmentieren – auf Original-Labels
y_train_original = le.inverse_transform(y_train)
X_aug, y_aug = augment_data(X_train, y_train_original)
y_aug_enc = le.transform(y_aug)

# 🔀 Kombinieren
X_train_combined = np.concatenate([X_train, X_aug])
y_train_combined = np.concatenate([y_train, y_aug_enc])

# 5️⃣ Modell bauen
model = models.Sequential([
    layers.Conv1D(64, 3, activation='relu', input_shape=X_train.shape[1:]),
    layers.Conv1D(64, 3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Dropout(0.3),
    layers.LSTM(64),
    layers.Dropout(0.3),
    layers.Dense(100, activation='relu'),
    layers.Dense(len(le.classes_), activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 6️⃣ Callback für Checkpoints
checkpoint_cb = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, "global_epoch_{epoch:02d}.keras"),
    save_best_only=True, save_weights_only=False, verbose=1
)

# 7️⃣ Training
history = model.fit(
    X_train_combined, y_train_combined,
    validation_data=(X_val, y_val),
    epochs=50, batch_size=64,
    callbacks=[checkpoint_cb]
)

# 8️⃣ Speichern des finalen Modells
model.save(os.path.join(final_model_dir, "global_final.keras"))

# 9️⃣ Evaluation
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
print(f"\n✅ Validation Accuracy: {val_acc:.2f}")

# 🔮 Vorhersagen
y_pred_probs = model.predict(X_val, verbose=0)
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# 📊 Report + Matrix
labels_present = unique_labels(y_val, y_pred_classes)
report_text = classification_report(
    y_val, y_pred_classes,
    labels=labels_present,
    target_names=[le.classes_[i] for i in labels_present]
)
print("\n📄 Classification Report:\n")
print(report_text)

# 💾 Speichern als Text
with open(os.path.join(report_dir, "global_model_classification_report.txt"), "w") as f:
    f.write(report_text)

# 📈 Confusion Matrix
cm = confusion_matrix(y_val, y_pred_classes, labels=labels_present)
disp = ConfusionMatrixDisplay(cm, display_labels=[le.classes_[i] for i in labels_present])
fig, ax = plt.subplots(figsize=(8, 6))
disp.plot(ax=ax, xticks_rotation=45)
plt.title("Confusion Matrix – Global Model Validation")

# 💾 Speichern als PNG
plt.savefig(os.path.join(report_dir, "global_model_confusion_matrix.png"))
plt.show()


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.utils.multiclass import unique_labels

# 💾 Speicherpfade anlegen
checkpoint_dir = "/content/drive/MyDrive/mtb_project/checkpoints_global"
os.makedirs(checkpoint_dir, exist_ok=True)
final_model_dir = "/content/drive/MyDrive/mtb_project/final_models_global"
os.makedirs(final_model_dir, exist_ok=True)

def augment_data(X, y, noise_std=0.01, time_shift_max=5):
    X_aug = []
    y_aug = []

    for xi, yi in zip(X, y):
        # 1. Leichtes Rauschen hinzufügen
        noisy = xi + np.random.normal(0, noise_std, xi.shape)

        # 2. Zufälliges Zeitverschieben
        shift = np.random.randint(-time_shift_max, time_shift_max)
        shifted = np.roll(noisy, shift, axis=0)

        # Optional: Padding beim Rollen (statt wrap-around)
        if shift > 0:
            shifted[:shift, :] = 0
        elif shift < 0:
            shifted[shift:, :] = 0

        X_aug.append(shifted)
        y_aug.append(yi)

    return np.array(X_aug), np.array(y_aug)

t

X_train_aug, y_train_aug = augment_timeseries(X_train, y_train, augment_factor=2)

# Augmentieren
X_aug, y_aug = augment_data(X_train, y_train)

# Kombinieren mit Originaldaten
X_train_combined = np.concatenate([X_train, X_aug])
y_train_combined = np.concatenate([y_train, y_aug])

# 1. Daten vorbereiten: alle Sessions zusammenführen


# 2. Labels encoden (wichtig für Modell)
le = LabelEncoder()
y_all_enc = le.fit_transform(y_all)





# 3. Trainings- und Validierungsdaten splitten (z.B. 90% Training, 10% Validation)
X_train, X_val, y_train, y_val = train_test_split(
    X_all, y_all_enc, test_size=0.1, random_state=42, stratify=y_all_enc
)

# Nachdem du X_train und y_train_enc oder y_train_binary hast

X_train_aug, y_train_aug = augment_timeseries(X_train, y_train, augment_factor=2)

# Augmentieren
X_aug, y_aug = augment_data(X_train, y_train)

# Kombinieren mit Originaldaten
X_train_combined = np.concatenate([X_train, X_aug])
y_train_combined = np.concatenate([y_train, y_aug])

# 4. Modell definieren (CNN + LSTM Architektur)
model = models.Sequential([
    layers.Conv1D(64, 3, activation='relu', input_shape=X_train.shape[1:]),
    layers.Conv1D(64, 3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Dropout(0.3),
    layers.LSTM(64),
    layers.Dropout(0.3),
    layers.Dense(100, activation='relu'),
    layers.Dense(len(le.classes_), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 5. Callback für Checkpoints während Training
checkpoint_cb = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, "global_epoch_{epoch:02d}.keras"),
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

# 6. Training starten
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=64,
    callbacks=[checkpoint_cb]
)

# 7. Finale Modell speichern
model.save(os.path.join(final_model_dir, "global_final.keras"))

# 8. Evaluation auf Validierungsdaten
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
print(f"\n✅ Validation Accuracy: {val_acc:.2f}")

# 9. Vorhersagen für Validierungsdaten erzeugen
y_pred_probs = model.predict(X_val, verbose=0)
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# 10. Berechne vorhandene Labels (für Report & Matrix)
labels_present = unique_labels(y_val, y_pred_classes)

# 11. Klassifikationsbericht ausgeben
print("\nClassification Report on Validation Set:")
print(classification_report(
    y_val,
    y_pred_classes,
    labels=labels_present,
    target_names=[le.classes_[i] for i in labels_present]
))

# 12. Confusion Matrix visualisieren
cm = confusion_matrix(y_val, y_pred_classes, labels=labels_present)
disp = ConfusionMatrixDisplay(cm, display_labels=[le.classes_[i] for i in labels_present])
disp.plot(xticks_rotation=45)
plt.title("Confusion Matrix – Global Model Validation")
plt.show()


NameError: name 'augment_timeseries' is not defined

#🧾 Zusammenfassung Modeltraining
##📊 1. Durchschnittliche Accuracy über alle Sessions:

In [ ]:
#📊 1. Durchschnittliche Accuracy über alle Sessions:

mean_acc = np.mean(all_accuracies)
print("\n📈 Zusammenfassung:")
for i, acc in enumerate(all_accuracies):
    print(f"  Session {i+1}: Accuracy = {acc:.2f}")
print(f"\n✅ Durchschnittliche Test-Accuracy über alle Sessions: {mean_acc:.2f}")


NameError: name 'all_accuracies' is not defined

##📁 2. CSV speichern (falls nicht schon vorhanden):

In [ ]:
#📁 2. CSV speichern (falls nicht schon vorhanden):

import pandas as pd

df_summary = pd.DataFrame(accuracy_summary)
df_summary['Session'] = [f"Session_{i+1}" for i in range(len(all_accuracies))]
df_summary.to_csv("/content/drive/MyDrive/mtb_project/session_accuracy_report.csv", index=False)
print("✅ Bericht gespeichert unter: session_accuracy_report.csv")


✅ Bericht gespeichert unter: session_accuracy_report.csv


In [ ]:

all_accuracies = []
accuracy_summary = {'Session': [], 'Accuracy': []}

for test_idx in range(len(sessions_X)):
    sess_name = session_dirs[test_idx]
    print(f"\n📌 Teste auf Session (unbekannt): {sess_name} ({test_idx+1}/{len(sessions_X)})")

    # 8.1 Test-Daten definieren
    X_test = sessions_X[test_idx]
    y_test = sessions_y[test_idx]



    # 8.2 Train-Daten: alle anderen Sessions zusammenschneiden
    X_train = np.concatenate([x for i, x in enumerate(sessions_X) if i != test_idx])
    y_train = np.concatenate([y for i, y in enumerate(sessions_y) if i != test_idx])

    # 8.3 Label-Encoding (fit auf Trainingsdaten, transform auf beides)
    le = LabelEncoder()
    y_train_enc = le.fit_transform(y_train)
    y_test_enc = le.transform(y_test)

    # 8.4 Modell-Definition: CNN + LSTM
    model = models.Sequential([
        layers.Conv1D(64, 3, activation='relu', input_shape=X_train.shape[1:]),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(0.3),
        layers.LSTM(64),
        layers.Dropout(0.3),
        layers.Dense(100, activation='relu'),
        layers.Dense(len(le.classes_), activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # 8.5 Training (mit 10 % Validierungssplit aus Trainingsdaten)

    from tensorflow.keras.callbacks import ModelCheckpoint
    import os

    # 1. 📁 Sicherstellen, dass Speicherort existiert
    checkpoint_dir = "/content/drive/MyDrive/mtb_project/checkpoints"
    os.makedirs(checkpoint_dir, exist_ok=True)

    # 2. 🎯 Callback definieren
    checkpoint_cb = ModelCheckpoint(
        filepath = os.path.join(checkpoint_dir, f"{sess_name}_epoch_{{epoch:02d}}.keras"),
        save_best_only=False,         # du bekommst jedes Epoch-Modell
        save_weights_only=False,      # speichert das ganze Modell, nicht nur Gewichte
        verbose=1
    )

# 3. 🚀 Training starten (Callback hinzufügen!)


    # mit checkpoints zum speichern
    history = model.fit(
        X_train, y_train_enc,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=64,
        callbacks=[checkpoint_cb] #-> automatische zwischenspeicherung
    )

    # 8.6 Evaluation auf Test-Session
    test_loss, test_acc = model.evaluate(X_test, y_test_enc, verbose=0)
    print(f"✅ Test-Accuracy für {sess_name}: {test_acc:.2f}")
    all_accuracies.append(test_acc)
    accuracy_summary['Session'].append(sess_name)
    accuracy_summary['Accuracy'].append(test_acc)

    # 8.7 Klassifikationsbericht & Confusion Matrix
    # 🔍 Report berechnen
    print("\nKlassifikationsbericht:")
    report_text = classification_report(
        y_test_enc,
        y_pred_classes,
        target_names=[le.classes_[i] for i in labels_present],
        labels=labels_present
    )
    print(report_text)

    # 💾 Bericht speichern (Textdatei)
    os.makedirs("/content/drive/MyDrive/mtb_project/reports", exist_ok=True)
    with open(f"/content/drive/MyDrive/mtb_project/reports/{sess_name}_classification_report.txt", "w") as f:
        f.write(report_text)

    # 📊 Confusion Matrix anzeigen & speichern
    cm = confusion_matrix(y_test_enc, y_pred_classes, labels=labels_present)
    fig, ax = plt.subplots(figsize=(8, 6))
    disp = ConfusionMatrixDisplay(cm, display_labels=[le.classes_[i] for i in labels_present])
    disp.plot(ax=ax, xticks_rotation=45)
    plt.title(f"Confusion Matrix – {sess_name}")
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/mtb_project/reports/{sess_name}_confusion_matrix.png")
    plt.show()



📌 Teste auf Session (unbekannt): Session_01 (1/12)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
330/330 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.5717 - loss: 1.0202
Epoch 1: saving model to /content/drive/MyDrive/mtb_project/checkpoints/Session_01_epoch_01.keras
330/330 ━━━━━━━━━━━━━━━━━━━━ 18s 45ms/step - accuracy: 0.5718 - loss: 1.0200 - val_accuracy: 0.5855 - val_loss: 0.9273
Epoch 2/50
329/330 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.6225 - loss: 0.8839
Epoch 2: saving model to /content/drive/MyDrive/mtb_project/checkpoints/Session_01_epoch_02.keras
330/330 ━━━━━━━━━━━━━━━━━━━━ 20s 42ms/step - accuracy: 0.6225 - loss: 0.8838 - val_accuracy: 0.6175 - val_loss: 0.8609
Epoch 3/50
330/330 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.6429 - loss: 0.8232
Epoch 3: saving model to /content/drive/MyDrive/mtb_project/checkpoints/Session_01_epoch_03.keras
330/330 ━━━━━━━━━━━━━━━━━━━━ 14s 42ms/step - accuracy: 0.6429 - loss: 0.8232 - val_accuracy: 0.6352 - val_loss: 0.8286
Epoch 4/50
329/330 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6582 - loss: 0.7952
E

ValueError: Found input variables with inconsistent numbers of samples: [1997, 2311]

In [ ]:
os.makedirs("/content/drive/MyDrive/mtb_project/reports", exist_ok=True)


In [ ]:
from sklearn.metrics import classification_report
import io

# Klassifikationsbericht als Text speichern
report_text = classification_report(y_test_enc, y_pred_classes, target_names=le.classes_)
with open(f"/content/drive/MyDrive/mtb_project/reports/{sess_name}_classification_report.txt", "w") as f:
    f.write(report_text)



#confusion matrix als bild speicheern
fig, ax = plt.subplots(figsize=(8,6))
disp = ConfusionMatrixDisplay(cm, display_labels=le.classes_)
disp.plot(ax=ax, xticks_rotation=45)
plt.title(f"Confusion Matrix – {sess_name}")
plt.tight_layout()
plt.savefig(f"/content/drive/MyDrive/mtb_project/reports/{sess_name}_confusion_matrix.png")
plt.close()


NameError: name 'y_test_enc' is not defined

In [ ]:
summary = {'Session': [], 'Accuracy': []}
for i, acc in enumerate(all_accuracies):
    summary['Session'].append(f"Session_{i+1}")
    summary['Accuracy'].append(acc)

df_summary = pd.DataFrame(summary)
df_summary.to_csv("session_accuracy_report.csv", index=False)
print("📁 Bericht gespeichert als session_accuracy_report.csv")

NameError: name 'all_accuracies' is not defined

🧾 Zusammenfassung nach dem Training:

Nach der Schleife kannst du am Ende folgendes hinzufügen, um einen Bericht zu erzeugen:

📊 Bonus: CSV speichern (optional)

In [ ]:

# Durchschnittliche Accuracy über alle Sessions
mean_acc = np.mean(all_accuracies)
print("\n📈 Zusammenfassung:")
for i, acc in enumerate(all_accuracies):
    print(f"  Session {i+1}: Accuracy = {acc:.2f}")
print(f"\n✅ Durchschnittliche Test-Accuracy über alle Sessions: {mean_acc:.2f}")



#📊 Bonus: CSV speichern (optional)

import pandas as pd

summary = {'Session': [], 'Accuracy': []}
for i, acc in enumerate(all_accuracies):
    summary['Session'].append(f"Session_{i+1}")
    summary['Accuracy'].append(acc)

df_summary = pd.DataFrame(summary)
df_summary.to_csv("session_accuracy_report.csv", index=False)
print("📁 Bericht gespeichert als session_accuracy_report.csv")




NameError: name 'all_accuracies' is not defined

#💾 II. Modell speichern & später wieder laden (z. B. nach Training)

##🔐 Speichern mit TensorFlow/**Keras**

In [ ]:
# Nach dem Training:
#speichert mit timestamp
#model.save("SenseCap_Eventdetection_Model.keras")  # speichert nur in colab kurzzeitig
import time
timestamp = time.strftime("%Y%m%d-%H%M%S")
model.save(f"/content/drive/MyDrive/ML-MTB-Modell/ML-Model_trained/SenseCap_Model_{timestamp}.keras")



##🔄 Laden

Das speichert das gesamte Modell inkl. Architektur, Gewichten und Optimizer-Zustand –exakt da weitermachen, wo man aufgehört hast.

In [ ]:


from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/ML-MTB-Modell/ML-Model_trained/SenseCap_Eventdetection_Model1.keras")


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 24 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
all_accuracies = []

for i in range(len(test_sessions_X)):
    X_test = test_sessions_X[i]
    y_test = test_sessions_y[i]

    _, acc = model.evaluate(X_test, y_test, verbose=0)
    all_accuracies.append(acc)


NameError: name 'test_sessions_X' is not defined

In [ ]:

# Durchschnittliche Accuracy über alle Sessions
mean_acc = np.mean(all_accuracies)
print("\n📈 Zusammenfassung:")
for i, acc in enumerate(all_accuracies):
    print(f"  Session {i+1}: Accuracy = {acc:.2f}")
print(f"\n✅ Durchschnittliche Test-Accuracy über alle Sessions: {mean_acc:.2f}")



#📊 Bonus: CSV speichern (optional)

import pandas as pd

summary = {'Session': [], 'Accuracy': []}
for i, acc in enumerate(all_accuracies):
    summary['Session'].append(f"Session_{i+1}")
    summary['Accuracy'].append(acc)

df_summary = pd.DataFrame(summary)
df_summary.to_csv("session_accuracy_report.csv", index=False)
print("📁 Bericht gespeichert als session_accuracy_report.csv")





📈 Zusammenfassung:

✅ Durchschnittliche Test-Accuracy über alle Sessions: nan
📁 Bericht gespeichert als session_accuracy_report.csv
